In [1]:
import numpy as np
from keras import backend as K
import tensorflow as tf
from keras.models import load_model
from scipy.ndimage import zoom, rank_filter, find_objects
import json
import copy
from utils import get_dicoms_from_folder, get_uid, getBase, get_blob_service, save_dcms, get_dicom_from_storage, insert_prediction, get_container_and_queue, push_message, get_vol_from_dcms

Using TensorFlow backend.
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Programas\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Fu

In [2]:
from skimage.measure import label, regionprops

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
K.set_session(tf.Session(config=config))

In [4]:
data = {"uuid": '6150a4eb-c677a15d-905434ad-527a3c5e', "company_id": 1}
uuid = data['uuid']
company_id = data['company_id']

In [5]:
dbConectionString = 'DRIVER=ODBC Driver 17 for SQL Server;SERVER=indiradb.database.windows.net;DATABASE={0};UID=indira;PWD=1ndir42019*'
container_security = 'indira-security'

blobStorageConnectionString = "DefaultEndpointsProtocol=https;AccountName=indirapacsadmin;AccountKey=h7KcU6C6Es/7bu2jyLIZKipCbw1397VUtQYh4GguNUsL3ISerjaPsLWZsmbnNYGifJHlVjz0djA+p8HlmPxCMA==;EndpointSuffix=core.windows.net"
container_name = "iastorage"

serviceBusConnectionString = 'Endpoint=sb://indira-bus.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=N356PDczXPj4xo7sLDbVJckSjAyWHpy0xmrwx8iw9V8='

container_db_name, queue_name = get_container_and_queue(company_id, dbConectionString.format(container_security))

if container_db_name == None or queue_name == None:
    raise Exception("the db container was not found")

In [6]:
segmentation_model_path = './segmentation_model.hdf5'
FPR_model_path = './FPR_model.hdf5'
cancer_model_path = './cancer_model.hdf5'
lobes_model_path = './lobes_model.hdf5'

In [7]:
segmentation_model = load_model(segmentation_model_path)
FPR_model = load_model(FPR_model_path)
cancer_model = load_model(cancer_model_path)
lobes_model = load_model(lobes_model_path)

Instructions for updating:
Colocations handled automatically by placer.


D:\Programas\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
############ utils lung CA #####################
def get_coods_matrix(start, stop):

    z = np.arange(start[0], stop[0], 64)
    z[len(z) - 1] = stop[0] - 64

    y = np.arange(start[1], stop[1], 64)
    y[len(y) - 1] = stop[1] - 64

    x = np.arange(start[2], stop[2], 64)
    x[len(x) - 1] = stop[2] - 64

    indexes = np.array(np.meshgrid(z, y, x))
    indexes = indexes.reshape(3, -1).T

    return indexes

def index_to_slice(start, thickness):
    slices = []
    for i in start:
        t = slice(i, i + thickness)
        slices.append(t)
    return slices

def normalize(npzarray):
    maxHU = 400.
    minHU = -1000.
    npzarray = (npzarray - minHU) / (maxHU - minHU)
    npzarray = np.clip(npzarray, 0, 1)
    return npzarray

def make_predictions_fom_vol(vol, indexes):
    try:
        thickness = 64
        pred = np.zeros((*vol.shape, 2))
        for start in indexes:
            slices = index_to_slice(start, thickness)
            cube = vol[slices]
            temp = segmentation_model.predict(cube[np.newaxis, ..., np.newaxis]) # *cube.shape, 1
            pred[slices] = temp[0]
    except Exception as e:
        print(str(e))
        raise('error al hacer una de las predicciones de los nodulos' + str(e))
    return pred

def get_start_stop(mask, margin = 32):
    limits = find_objects(mask != 0)[0]
    start = []
    stop = []
    for i, limit in enumerate(limits):
        begin = limit.start - 32 if limit.start >= 32 else 0
        end = limit.stop + 32 if limit.stop + 32 < mask.shape[i] else mask.shape[i]
        start.append(begin)
        stop.append(end)
    return start, stop

def get_nodule_segmentation(vol, lobes_mask):
    #start = (0, 0, 0)
    #stop = vol.shape
    start, stop = get_start_stop(lobes_mask)
    indexes = get_coods_matrix(start, stop)
    pred = make_predictions_fom_vol(normalize(vol), indexes).argmax(-1)
    return pred

def FPR(patch):
    vol_size = np.array([30, 30, 30])
    patch = normalize(patch)
    patch = zoom(patch, vol_size/np.array(patch.shape), order=0)[np.newaxis, ..., np.newaxis]
    tag = FPR_model.predict(patch)[0] > 0.64
    return tag

def refine_segmentation(vol, segmentation):
    labeled_segmentation = label(segmentation)
    props_pred = regionprops(labeled_segmentation)
    
    for prop in props_pred:
        min_slice, min_row, min_col, max_slice, max_row, max_col = prop.bbox
        patch = vol[min_slice:max_slice, min_row:max_row, min_col:max_col]
        is_tp = FPR(patch)
        if not is_tp:
            segmentation[labeled_segmentation == prop.label] = 0
            
    return segmentation

In [9]:
def process_lobe_prediction(pred, new_shape, apply_rank_filter=True, resize=True):
    pred = pred.reshape(128,128,128,6)
    mask = pred.argmax(-1)
    if resize:
        scale = np.array(new_shape)/np.array(mask.shape)
        mask = zoom(mask, scale, order=0)
    if apply_rank_filter:
        mask =  rank_filter(mask, rank=165, size=7)
    return mask

def mask_to_dcms(dcms, arr, sort=True, sort_by="InstanceNumber"):
    max_value = arr.max()
    bits_stored = len(bin(max_value))-2
    
    series_uid = get_uid()
    dcm_base = getBase(dcms[0], series_uid, 'Segmented Lobe', wc='3', ww='6', bits = 8, bits_stored=bits_stored, intercept='0', slope='1')
    arr = arr.astype(np.uint8)
    dcms_new = []
    
    for pixel_array, dcm in zip(arr, dcms):
        instance_uid = get_uid()
        
        dcm_base.add_new(0x00080018, 'UI', instance_uid) # SOP Instance UID
        dcm_base.add_new(0x00200013, 'IS', dcm.InstanceNumber) # Instance Number
        dcm_base.add_new(0x00201041, 'DS', dcm.SliceLocation) # Slice Location
        dcm_base.add_new(0x00200032, 'DS', dcm.ImagePositionPatient) # Image Position (Patient)
        dcm_base.add_new(0x7fe00010, 'OW', pixel_array.tobytes()) # Pixel Data

        dcms_new.append(copy.deepcopy(dcm_base))
        #dcm_base.save_as('temp/{}.dcm'.format(instance_uid))

    dcms_new.sort(key=lambda x: float(x.get(sort_by)))

    return dcms_new

def segment_and_save_lobes_from_dcms(dcms, company_id, apply_rank_filter=True, resize=True):
    vol, _ = get_vol_from_dcms(dcms, sort_by="SliceLocation")
    new_shape = np.array([128, 128, 128])
    old_shape = vol.shape
    vol = zoom(vol, new_shape/np.array(vol.shape), order=0)
    vol = normalize(vol)
    pred = lobes_model.predict(vol.reshape(1, 128, 128, 128, 1))
    mask = process_lobe_prediction(
        pred, old_shape, apply_rank_filter, resize)
    
    lobes_dcms = mask_to_dcms(dcms, mask, sort=True, sort_by="InstanceNumber")
    _, folder_path = save_dcms(lobes_dcms, blobStorageConnectionString, container_name, company_id)[0]

    push_message(json.dumps({'path': folder_path, 'type': "PullDcms"}), serviceBusConnectionString, queue_name)
    return mask, folder_path

In [25]:
def get_cancer_prediction(patch):
    vol_size = np.array([40, 40, 40])
    patch = normalize(patch)
    patch = zoom(patch, vol_size/np.array(patch.shape), order=0)[np.newaxis, ..., np.newaxis]
    tag = cancer_model.predict(patch)[0]
    return tag

def get_diameter(bbox, spacing):
    min_slice, min_row, min_col, max_slice, max_row, max_col = prop.bbox
    x_diameter = max_col - min_col
    y_diameter = max_row - min_row
    
    if x_diameter >= y_diameter:
        return x_diameter * spacing[2]
    else:
        return y_diameter * spacing[1]

In [10]:
dcms = get_dicoms_from_folder(uuid, container_name, blobStorageConnectionString)
vol, spacing = get_vol_from_dcms(dcms)

In [14]:
lobes_mask, folder_path = segment_and_save_lobes_from_dcms(dcms, company_id)

text: {"path": "1/1.2.826.0.1.3680043.10.455.108.121238227777.56678378762199502542", "type": "PullDcms"}


In [18]:
segmentation = get_nodule_segmentation(vol, lobes_mask)

D:\Programas\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
D:\Programas\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [19]:
segmentation = refine_segmentation(vol, segmentation)

D:\Programas\Anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [20]:
pred_final = label(segmentation)

In [21]:
np.unique(pred_final)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64)

In [62]:
props_pred = regionprops(pred_final)
measures = {}
to_skip = []

In [63]:
i = 0
for prop in props_pred:
    # determina en que lobulo esta el nodulo
    lobe_values = lobes_mask[pred_final == prop.label]
    vals, counts = np.unique(lobe_values, return_counts=True)
    counts = counts[vals != 0]
    vals = vals[vals != 0]

    if len(vals) == 0:
        to_skip.append(prop.label)
        continue
    
    lobe = int(vals[counts == counts.max()][0])
    
    min_slice, min_row, min_col, max_slice, max_row, max_col = prop.bbox
    patch = vol[min_slice:max_slice, min_row:max_row, min_col:max_col]
    
    cancer_proba = round(get_cancer_prediction(patch)[1]*100, 2)
    
    diameter = get_diameter(prop.bbox, spacing)

    measures[str(i)] = {
        'center': (np.array(prop.centroid)).round().astype(np.int).tolist(),
        'cords': {},
        'type': 1 if cancer_proba > 0.5 else 2,
        'lobe': lobe,
        'probability': cancer_proba,
        'diameter': diameter
    }
    
    slices, cords = np.where((pred_final == prop.label).reshape(vol.shape[0], -1))
    for cut in slices:
        measures[str(prop.label)]['cords'][str(cut)] = cords[slices == cut].tolist()
    i += 1

In [ ]:
insert_prediction(dcm.StudyInstanceUID, dcm.SeriesInstanceUID, dcm.SOPInstanceUID, json.dumps(data), 2, 5, dbConectionString.format(container_db_name))